ch10_embedding_221101

## 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/산대특/deeplearning/ch10/data

/content/drive/MyDrive/Colab Notebooks/산대특/deeplearning/ch10/data


## 패스트텍스트
- 패스트텍스트(FastText)는 워드투벡터의 단점을 보완하고자 페이스북에서 개발한 임베딩 알고리즘
- 교재 p414 페이지 확인

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ko.vec

--2022-11-01 04:58:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ko.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2323010599 (2.2G) [binary/octet-stream]
Saving to: ‘wiki.ko.vec’

wiki.ko.vec         100%[===================>]   2.16G  37.6MB/s    in 50s     

2022-11-01 04:59:14 (44.4 MB/s) - ‘wiki.ko.vec’ saved [2323010599/2323010599]



## 글로브
- 횟수 기반의 LSA와 예측 기반의 워드투벡터 단점을 보완하기 위한 모델
- 교재 p416

In [4]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2022-11-01 04:59:14--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-01 04:59:14--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.98MB/s    in 2m 43s  

2022-11-01 05:01:58 (5.03 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [6]:
!unzip 'IMDB Dataset.zip'

Archive:  IMDB Dataset.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


### 원핫 인코딩 적용

In [7]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/산대특/deeplearning/ch10/data/"

class2 = pd.read_csv(DATA_PATH + "class2.csv")
class2.head()

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,CID000,C,CIRC,N,535.0,475.0,192.0
1,1,CID001,A,CIRA,N,433.0,268.0,58.0
2,2,CID002,A,CIRA,I,NaN,NaN,NaN
3,3,CID003,C,CIRC,B,NaN,NaN,NaN
4,4,CID004,F,CIRF,I,488.0,145.0,29.0


In [8]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

# pd.get_dummies()

train_x = label_encoder.fit_transform(class2['class'])
train_x

array([1, 0, 0, 1, 2, 1])

In [10]:
train_x = pd.get_dummies(class2)
train_x

,Unnamed: 0,x,y,r,id_CID000,id_CID001,id_CID002,id_CID003,id_CID004,id_CID005,tissue_A,tissue_C,tissue_F,class_CIRA,class_CIRC,class_CIRF,class2_B,class2_I,class2_N
0,0,535.0,475.0,192.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1
1,1,433.0,268.0,58.0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1
2,2,NaN,NaN,NaN,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0
3,3,NaN,NaN,NaN,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0
4,4,488.0,145.0,29.0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0
5,5,532.0,199.0,21.0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0


- 자연어 처리를 할 때는 원-핫 인코딩 사용 X
- 직교를 이룬다 --> 서로 독립적인 관계가 된다.
- 대안을 찾자 : 워드투벡터, 글로브, 패스트텍스트 등을 사용한다.

### 횟수 기반 임베딩
- CountVectorizer, 교재 (p400)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [12]:
vect.transform(["you will never get any chance."]).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [13]:
vect = CountVectorizer(stop_words=["and", "is", "please", "this"]).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

- TF-IDF, 교재 (p402)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
doc = ['I like machine learning', 'I love deep learning', 'I run everyday']
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)
print ('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), '행렬을 만들었습니다.')
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


### 예측 기반 임베딩
- 워드 투 벡터
  + 신경망 알고리즘의 하나
- and I love this one
- one을 입력 --> 출력층 나머지 4개 단어 구성
- 각 단어 간의 코사인 유사도를 통해 해당 단어의 의미를 수치화 한다.

In [16]:
!ls

 class2.csv	     glove.6B.300d.txt	'IMDB Dataset.zip'   test.csv
 example.txt	     glove.6B.50d.txt	 peter.txt	     train.csv
 glove.6B.100d.txt   glove.6B.zip	 spam.csv	     wiki.ko.vec
 glove.6B.200d.txt  'IMDB Dataset.csv'	 spa.txt


In [22]:
from nltk.tokenize import sent_tokenize,word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import gensim
from gensim.models import Word2Vec

sample =open("peter.txt", "r", encoding="UTF-8")
s=sample.read()
s

'Once upon a time in London, the darlings went out to a dinner party leaving their three children Wendy, Jhon, and Michael at home. After Wendy had tucked her younger brothers Jhon and Michael to bed, she went to read a book. She heard a boy sobbing outside her window. He was flying. There was little fairy fluttering around him. Wendy opened the window to talk to him.\n\n“Hello! Who are you? Why are you crying ”, wendy asked him. “My name is Peter Pan. my shadow wouldn’t stock to me.”, He replied. She asked him to come in. Peter agreed and came inside the room. Wendy took his shadow and sewed it to his shoe tips.\n\nNow his shadow followed him wherever Peter Pan went! He was delighted and asked Wendy “Why don’t you come with me to my home. The Neverland. I lived there with my fairy Tinker Bell.” Wendy ? “Oh! What a wonderful idea! Let me wake up John and Micheal too. Could you teach us how to fly?”. “Yes! Of course! Get them we will all fly together.” Peter Pan replied\n\nAnd so it was

In [24]:
# 줄바꿈을 공백으로 전환
f = s.replace("\n"," ")
data = []
for i in sent_tokenize(f):
  temp =[]
  for j in word_tokenize(i):
    temp.append(j.lower())
  data.append(temp)

data  

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

### 단어간 유사성을 확인

In [31]:
# sg=0, CBOW 알고리즘 선택
# sg=1, skip-gram 알고리즘을 선택한다.
model1 = gensim.models.Word2Vec(data, min_count=1,
                               size=100,window=5,sg=0)

print("Cosine similarity between 'peter' " +
                 "'hook' - CBOW : ", 
      model1.similarity('peter', 'hook')) 

Cosine similarity between 'peter' 'hook' - CBOW :  0.083885044


In [33]:
print("Cosine similarity between 'peter' " +
                 "'wendy' - CBOW : ", 
      model1.similarity('peter', 'wendy')) 

Cosine similarity between 'peter' 'wendy' - CBOW :  0.10191567


In [32]:
# sg=0, CBOW 알고리즘 선택
# sg=1, skip-gram 알고리즘을 선택한다.
model2 = gensim.models.Word2Vec(data, min_count=1,
                               size=100,window=5,sg=1)

print("Cosine similarity between 'peter' " +
                 "'hook' - Skip Gram : ", 
      model2.similarity('peter', 'hook')) 

Cosine similarity between 'peter' 'hook' - Skip Gram :  0.55354315


In [34]:
print("Cosine similarity between 'peter' " +
                 "'wendy' - Skip Gram : ", 
      model2.similarity('peter', 'wendy')) 

Cosine similarity between 'peter' 'wendy' - Skip Gram :  0.37650046


### 패스트텍스트
- 워드 투 벡터 : 사전에 모르는 단어는 학습이 어려움
- 워드 투 벡터의 단점을 보완함.
- 소파 위에 있는 고양이가 낮잠을 잔다.
  + 워드 투 벡터 --> 문장 1개만 사용
  + N-그램 --> 세트를 3개 정도

In [39]:
from gensim.test.utils import common_texts 
from gensim.models import FastText 

corpus_fname = 'peter.txt'
corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r', encoding='utf-8').readlines()]
model = FastText(corpus, size=4, window=3, min_count = 1, iter = 10)

sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

-0.5548141


In [41]:
sim_score = model.wv.similarity('peter', 'hook')
print(sim_score)

-0.1171107


## 한글 모델 사용
- 사전 훈련된 모델을 호출하자.

In [42]:
!ls

 class2.csv	     glove.6B.300d.txt	'IMDB Dataset.zip'   test.csv
 example.txt	     glove.6B.50d.txt	 peter.txt	     train.csv
 glove.6B.100d.txt   glove.6B.zip	 spam.csv	     wiki.ko.vec
 glove.6B.200d.txt  'IMDB Dataset.csv'	 spa.txt


In [43]:
from __future__ import print_function
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format("wiki.ko.vec")
print(model_kr)

In [44]:
find_similar_to = '노력'

for similar_word in model_kr.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word: 노력함, Similarity: 0.80
Word: 노력중, Similarity: 0.75
Word: 노력만, Similarity: 0.72
Word: 노력과, Similarity: 0.71
Word: 노력의, Similarity: 0.69
Word: 노력가, Similarity: 0.69
Word: 노력이나, Similarity: 0.69
Word: 노력없이, Similarity: 0.68
Word: 노력맨, Similarity: 0.68
Word: 노력보다는, Similarity: 0.68


In [45]:
similarities = model_kr.wv.most_similar(positive=['동물', '육식동물'], negative=['사람'])
print(similarities)

[('초식동물', 0.7804122567176819), ('거대동물', 0.7547270059585571), ('육식동물의', 0.7547166347503662), ('유두동물', 0.7535113096237183), ('반추동물', 0.7470757961273193), ('독동물', 0.7466292381286621), ('육상동물', 0.746031641960144), ('유즐동물', 0.7450904846191406), ('극피동물', 0.7449344992637634), ('복모동물', 0.7424346208572388)]


In [46]:
similarities = model_kr.wv.most_similar(positive=['일본', '중국'], negative=['한국'])
print(similarities)

[('중화인민공하국', 0.6181370615959167), ('중화인민공화국', 0.5936577320098877), ('톈랴오', 0.586189329624176), ('톈중', 0.5725148916244507), ('대만', 0.5692499876022339), ('톈진의', 0.569195032119751), ('훙후', 0.5680252313613892), ('중국등', 0.5665101408958435), ('광저우시', 0.5654244422912598), ('랴오중카이', 0.5621193647384644)]


### 횟수/예측 기반 임베딩
- 횟수 기반 임베딩의 단점
- 예측 기반 임베딩의 단점
- Glove : 위 두개의 알고리즘 단점을 보완.

In [47]:
!ls

 class2.csv	     glove.6B.300d.txt	'IMDB Dataset.zip'   test.csv
 example.txt	     glove.6B.50d.txt	 peter.txt	     train.csv
 glove.6B.100d.txt   glove.6B.zip	 spam.csv	     wiki.ko.vec
 glove.6B.200d.txt  'IMDB Dataset.csv'	 spa.txt


In [48]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = 'glove.6B.100d.txt'
word2vec_glove_file = get_tmpfile ('glove.6B.100d.word2vec.txt')
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [49]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
model.most_similar('bill')

[('legislation', 0.8072140216827393),
 ('proposal', 0.7306863069534302),
 ('senate', 0.7142540812492371),
 ('bills', 0.7044401168823242),
 ('measure', 0.6958035230636597),
 ('passed', 0.6906244158744812),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663139462471008)]

## p.419 트랜스포머 어텐션
- 자연어 알고리즘의 가장 중요한 핵심!
- 지금 현재
  + 트랜스포머 알고리즘 시대!
- 단어 번역
  + 새로운 단어, 다른 모든 단어와 비교해서 번역
  + 병렬구조로 처리 --> 속도도 향상 / 정확도도 향상
- 영향력
  + GPT-123 계열 
  + Bert 계열